In [2]:
#import libraries.

import cv2
import numpy  as np
from collections import deque
import time

In [3]:
#define things

points = deque(maxlen = 1000)

blackboard = np.zeros((512,512), np.uint8)


In [ ]:
#loading the save model
from keras.models import load_model
cnn_model = load_model('emnist_cnn_model.h5')

# Capturing the Writing

In [13]:
#starting the loop
cap = cv2.VideoCapture()   
while True:
    _, frame = cap.read()
    #flipping to get the correct orientation
    frame = cv2.flip(frame, 1)
    #hsv
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # lower mask (0-10)
    lower_red = np.array([0,50,50])
    upper_red = np.array([10,255,255])
    mask0 = cv2.inRange(hsv, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(hsv, lower_red, upper_red)
    
    # join my masks
    mask = mask0+mask1
    #red
    red=hsv.copy()
    red[np.where(mask==0)] = 0

    #red median blur
    red_median = cv2.medianBlur(red, 15)
    
    #find contours
    (contours, _) = cv2.cvtColor(red_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    try:
        if len(contours) > 0:
            contour_list = sorted(contours, key = cv2.contourArea, reverse = True)
            contour = contour_list[0]
            #find centroid
            M = cv2.moments(contour) #M will be a dict
            centroid_x = int(M["m10"]/M["m00"])
            centroid_y = int(M["m01"]/M["m00"])
            centroid = (centroid_x, centroid_y)
            #append centroid in points
            points.appendleft(centroid)
            #draw circle
            ((x,y), r) = cv2.minEnclosingCircle(contour)
            cv2.circle(frame, (int(x),int(y)), int(r), (0,255,0), 2)
    except ZeroDivisionError:
        pass
            
    for i in range(1, len(points)):
        cv2.line(frame, tuple(map(int,points[i])), tuple(map(int,points[i-1])), (0,0,0), 2)
        cv2.line(blackboard, tuple(map(int,points[i])), tuple(map(int,points[i-1])), (255,255,255), 8)

    cv2.imshow("frame", frame)
    cv2.imshow("blackboard", blackboard)
    
    if cv2.waitKey(1) &  0xFF == ord("q"):
        break

    
blackboard_median = cv2.medianBlur(blackboard, 15)
alphabet = cv2.resize(blackboard_median, (28,28))
alphabet = np.array(alphabet)
alphabet = alphabet.astype("float32")/255


    
points = deque(maxlen = 1000)
blackboard = np.zeros((512,512), np.uint8)

cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-ttbyx0jz\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [18]:
write()

KeyboardInterrupt: 